# Trabajo Práctico 1: **Clasificador de Recomendaciones Recreativas utilizando Procesamiento de Lenguaje Natural**

**Contexto:** Una persona dentro de un mes, se tomará 15 días de vacaciones en la playa. Sin embargo, se estima que durante al menos cuatro de esos días habrá lluvias, lo que podría limitar las actividades al aire libre. Para esos días de mal clima, se propone una solución que facilite la recreación en función del estado de ánimo del día.

**Objetivo:** Desarrollar un programa de Procesamiento de Lenguaje Natural que, según el estado de ánimo del usuario, recomiende entre ver una película, jugar un juego de mesa o leer un libro (o varias opciones para cada caso). Para ello, deberá construir un clasificador que categorice el estado de ánimo del usuario. Luego sugerir el conjunto de recomendaciones basada en una frase de preferencia ingresada por el usuario.


# Dataset y librerías

In [ ]:
!pip install spacy && python -m spacy download en_core_web_sm`

/bin/bash: -c: line 1: unexpected EOF while looking for matching ``'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [ ]:
!pip install transformers sentence_transformers
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.0 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=59aea88faccbbd2599f9e5c20870bb120b22bf62cad5bca80726c1172d83f688
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Succ

In [ ]:
!pip install gliner
from gliner import GLiNER

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import ast
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import nltk
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, util
import requests
from bs4 import BeautifulSoup
import csv
from googletrans import Translator
import joblib
import spacy


In [ ]:
# Funcion para traducir frases en castellano a ingles

translator = Translator()
def translate(text):
  translation = translator.translate(text, src='es', dest='en')
  return translation.text

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


***Pasos para la construcción del proyecto:***

**Clasificación del Estado de Ánimo:**

Utilice los conocimientos aprendidos en la Unidad 3 para desarrollar un clasificador a partir de un prompt con el que determine el estado de ánimo del usuario, el cual deberá categorizarse por ejemplo: "Alegre", "Melancólico" o "Ni fu ni fa".


# **Clasificador de estado de animo**

In [ ]:
# Se utilizara para entrenar el modelo el dataset Emotion de Kaggle.
# Este cuenta con frases en ingles asociadas a un estado de animo (6 en total)
# https://www.kaggle.com/datasets/parulpandey/emotion-dataset?resource=download

In [ ]:
archivo_csv = '/content/drive/MyDrive/NLP/emotion.csv'

# Labels (0, "triste"), (1, "alegre"),(2, "amor"),(3, "enojo"),(4, "miedo"),(5, "sorpresa")
# Se filtraran algunas emociones, se trabajara con "enojo", "alegre", "triste"

dataset = []
with open(archivo_csv, mode='r', encoding='utf-8') as archivo:
    lector = csv.reader(archivo)

    next(lector)

    for fila in lector:
        label = fila[1]
        emotion = fila[0]
        dataset.append((label, emotion))

In [ ]:
labels = ["triste", "alegre", "amor", "enojo", "miedo", "sorpresa"]

In [ ]:
# Balanceo del dataset
# el dataset esta muy desbalanceado, ya que la clase minoritaria es incluso casi 10 veces mas chica que la mayoritaria

contador = {
    "0": 0,
    "1": 0,
    "2": 0,
    "3": 0,
    "4": 0,
    "5": 0,
}

for indice, texto in dataset:
    contador[indice] += 1

for indice, cuenta in contador.items():
    print(f"Índice {labels[int(indice)]}: {cuenta} veces")

Índice triste: 5797 veces
Índice alegre: 6761 veces
Índice amor: 1641 veces
Índice enojo: 2709 veces
Índice miedo: 2373 veces
Índice sorpresa: 719 veces


In [ ]:
# Para abordar el desbalanceo se toma las siguientes decisiones:
# 1) No tener en cuenta la emocion sorpresa, amor y miedo
# 2) Tomar de las emociones restantes 2709 registros (cantidad de enojo)
# En este caso es preferible contar con menos registros en el dataset, pero que este se encuentre balanceado.

contador = {
    "0": 0,
    "1": 0,
    "3": 0,
}

dataset_filtered = []
for indice, texto in dataset:
    if indice != "5" and indice != "4" and indice != "2" and contador[indice] < 2709:
        dataset_filtered.append((indice, texto))
        contador[indice] += 1

In [ ]:
dataset = dataset_filtered

In [ ]:
# Modelo a entrenar
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# estados de animo a utilizar
labels = [(0, "triste"), (1, "alegre"),(3, "enojo"),]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Preparar X e y
X = [text.lower() for label, text in dataset]
y = [label for label, text in dataset]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Obtenemos los embeddings de BERT para los conjuntos de entrenamiento y prueba
X_train_vectorized = model.encode(X_train)
X_test_vectorized = model.encode(X_test)

# Creación y entrenamiento del modelo de Regresión Logística Multinomial
modelo_LR = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
modelo_LR.fit(X_train_vectorized, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [ ]:
# Guardado del modelo

# Guardar el modelo entrenado
joblib.dump(modelo_LR, 'modelo_LR.pkl')

['modelo_LR.pkl']

In [ ]:
# Evaluación del modelo de Regresión Logística
y_pred_LR = modelo_LR.predict(X_test_vectorized)
acc_LR = accuracy_score(y_test, y_pred_LR)
report_LR = classification_report(y_test, y_pred_LR, zero_division=1)

print("Precisión Regresión Logística:", acc_LR)
print("Reporte de clasificación Regresión Logística:\n", report_LR)

Precisión Regresión Logística: 0.7742927429274292
Reporte de clasificación Regresión Logística:
               precision    recall  f1-score   support

           0       0.75      0.73      0.74       561
           1       0.81      0.80      0.81       545
           3       0.76      0.80      0.78       520

    accuracy                           0.77      1626
   macro avg       0.77      0.78      0.77      1626
weighted avg       0.77      0.77      0.77      1626



In [ ]:
# Función para clasificar una frase y devolver la emoción
def clasificar_emocion(frase):
    emociones = {
      '0': "triste",
      '1': "alegre",
      '3': "enojo"
    }

    frase_traducida = translate(frase)
    frase_lower = frase_traducida.lower()
    frase_vectorizada = model.encode([frase_lower])
    new_predictions = modelo_LR.predict(frase_vectorizada)[0]
    emocion = emociones.get(new_predictions, "Emoción desconocida")

    print(f"Frase: '{frase}'")
    print(f"La emoción más presente hoy es: {emocion}\n")

In [ ]:
# Iterar en las frases de new_phrases y clasificar cada una
new_phrases = [
    "me siento con felicidad y entusiasmo",
    "Siento enojo, no tengo ganas de hacer nada",
    "Me siento inseguro",
    "siento feliz y con amor",
    "me siento triste, las cosas no me motivan",
    "hoy es un dia que me tiene irritado, no quiero hablar con nadie",
    "hoy me encuentro motivado para hacer nuevas cosas"
]

for frase in new_phrases:
    clasificar_emocion(frase)

Frase: 'me siento con felicidad y entusiasmo'
La emoción más presente hoy es: alegre

Frase: 'Siento enojo, no tengo ganas de hacer nada'
La emoción más presente hoy es: enojo

Frase: 'Me siento inseguro'
La emoción más presente hoy es: triste

Frase: 'siento feliz y con amor'
La emoción más presente hoy es: alegre

Frase: 'me siento triste, las cosas no me motivan'
La emoción más presente hoy es: triste

Frase: 'hoy es un dia que me tiene irritado, no quiero hablar con nadie'
La emoción más presente hoy es: enojo

Frase: 'hoy me encuentro motivado para hacer nuevas cosas'
La emoción más presente hoy es: alegre



# **Recomendador**

**Ingreso de Preferencias:**

Una vez determinado el estado de ánimo, el usuario deberá ingresar una frase que describa la temática que le gustaría explorar. Por ejemplo: "una historia de amor en la selva".


**Búsqueda de Opciones:**

El programa deberá comparar la frase ingresada por el usuario con diversas estructuras de texto provenientes de diferentes fuentes de datos utilizando los métodos aprendidos en clase.

Disponga de los siguientes datasets:

*   bgg_database.csv: Base de datos de juegos de mesa.

*   IMDB-Movie-Data.csv: Base de datos de películas.

*   Libros del Proyecto Gutenberg: Realice web scraping para conformar un dataset con información sobre los 1000 libros más populares del Proyecto Gutenberg. El enlace a utilizar es el siguiente: https://www.gutenberg.org/browse/scores/top1000.php#books-last1.

**Recomendaciones:**

Con base en el estado de ánimo del usuario y la frase ingresada, el programa deberá ofrecer recomendaciones pertinentes entre películas, juegos de mesa o libros. Utilice las herramientas de NLP aprendidas en las tres primeras unidades para lograr resultados coherentes y personalizados.


# Juegos

**Datasets**

In [ ]:
# Dataset de juegos de mesa bgg

bgg_df = pd.read_csv('/content/drive/MyDrive/NLP/bgg_database.csv')
bgg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              1000 non-null   int64  
 1   game_name         1000 non-null   object 
 2   game_href         1000 non-null   object 
 3   geek_rating       1000 non-null   float64
 4   avg_rating        1000 non-null   float64
 5   num_voters        1000 non-null   float64
 6   description       1000 non-null   object 
 7   yearpublished     1000 non-null   int64  
 8   minplayers        1000 non-null   int64  
 9   maxplayers        1000 non-null   int64  
 10  minplaytime       1000 non-null   int64  
 11  maxplaytime       1000 non-null   int64  
 12  minage            1000 non-null   int64  
 13  avgweight         1000 non-null   float64
 14  best_num_players  1000 non-null   object 
 15  designers         1000 non-null   object 
 16  mechanics         1000 non-null   object 
 

In [ ]:
bgg_df.head(3)

,rank,game_name,game_href,geek_rating,avg_rating,num_voters,description,yearpublished,minplayers,maxplayers,minplaytime,maxplaytime,minage,avgweight,best_num_players,designers,mechanics,categories
0,1,Brass: Birmingham,https://boardgamegeek.com/boardgame/224517/bra...,8.415,8.60,46836.0,Brass: Birmingham is an economic strategy game...,2018,2,4,60,120,14,3.8776,"[{'min': 3, 'max': 4}]","['Gavan Brown', 'Matt Tolman', 'Martin Wallace']","['Hand Management', 'Income', 'Loans', 'Market...","['Age of Reason', 'Economic', 'Industry / Manu..."
1,2,Pandemic Legacy: Season 1,https://boardgamegeek.com/boardgame/161936/pan...,8.377,8.53,53807.0,Pandemic Legacy is a co-operative campaign gam...,2015,2,4,60,60,13,2.8308,"[{'min': 4, 'max': 4}]","['Rob Daviau', 'Matt Leacock']","['Action Points', 'Cooperative Game', 'Hand Ma...","['Environmental', 'Medical']"
2,3,Gloomhaven,https://boardgamegeek.com/boardgame/174430/glo...,8.349,8.59,62592.0,Gloomhaven is a game of Euro-inspired tactica...,2017,1,4,60,120,14,3.9132,"[{'min': 3, 'max': 3}]",['Isaac Childres'],"['Action Queue', 'Action Retrieval', 'Campaign...","['Adventure', 'Exploration', 'Fantasy', 'Fight..."


In [ ]:
all_categories = bgg_df['categories'].apply(lambda x: eval(x) if isinstance(x, str) else x).tolist()

# Aplanar la lista de listas y obtener categorías únicas
unique_categories = set([category for sublist in all_categories for category in sublist])

# Convertir a lista y ordenar las categorías
unique_categories = sorted(list(unique_categories))

print("Categorías únicas:", unique_categories)

Categorías únicas: ['Abstract Strategy', 'Action / Dexterity', 'Adventure', 'Age of Reason', 'American Civil War', 'American Indian Wars', 'American Revolutionary War', 'American West', 'Ancient', 'Animals', 'Arabian', 'Aviation / Flight', 'Bluffing', 'Card Game', "Children's Game", 'City Building', 'Civil War', 'Civilization', 'Collectible Components', 'Comic Book / Strip', 'Deduction', 'Dice', 'Economic', 'Educational', 'Electronic', 'Environmental', 'Expansion for Base-game', 'Exploration', 'Fantasy', 'Farming', 'Fighting', 'Game System', 'Horror', 'Humor', 'Industry / Manufacturing', 'Mafia', 'Math', 'Mature / Adult', 'Maze', 'Medical', 'Medieval', 'Memory', 'Miniatures', 'Modern Warfare', 'Movies / TV / Radio theme', 'Murder/Mystery', 'Music', 'Mythology', 'Napoleonic', 'Nautical', 'Negotiation', 'Novel-based', 'Number', 'Party Game', 'Pike and Shot', 'Pirates', 'Political', 'Post-Napoleonic', 'Prehistoric', 'Print & Play', 'Puzzle', 'Racing', 'Real-time', 'Religious', 'Renaissanc

In [ ]:
# Clasificación de categorías según emociones
emotions_dict = {
    'enojo': [
        'American Civil War', 'American Indian Wars', 'American Revolutionary War',
        'Civil War', 'Modern Warfare', 'Vietnam War', 'Wargame', 'Political',
        'Mafia', 'Murder/Mystery', 'Horror'
    ],
    'alegría': [
        'Abstract Strategy', 'Adventure', 'Animals', 'Bluffing', 'Card Game',
        "Children's Game", 'City Building', 'Collectible Components', 'Economic',
        'Educational', 'Environmental', 'Exploration', 'Fantasy', 'Farming',
        'Humor', 'Memory', 'Miniatures', 'Party Game', 'Puzzle', 'Racing',
        'Real-time', 'Science Fiction', 'Space Exploration', 'Spies/Secret Agents',
        'Sports', 'Trivia', 'Video Game Theme', 'Zombies'
    ],
    'tristeza': [
        'Aviation / Flight', 'Age of Reason', 'Ancient', 'Arabian', 'Deduction',
        'Dice', 'Electronic', 'Expansion for Base-game', 'Fighting', 'Game System',
        'Math', 'Mature / Adult', 'Maze', 'Medical', 'Medieval', 'Mythology',
        'Napoleonic', 'Nautical', 'Negotiation', 'Novel-based', 'Number',
        'Pike and Shot', 'Pirates', 'Post-Napoleonic', 'Prehistoric',
        'Print & Play', 'Transportation', 'Travel', 'World War I',
        'World War II'
    ]
}

# Imprimir el diccionario de emociones
for emotion, categories in emotions_dict.items():
    print(f"{emotion}: {categories}\n")

enojo: ['American Civil War', 'American Indian Wars', 'American Revolutionary War', 'Civil War', 'Modern Warfare', 'Vietnam War', 'Wargame', 'Political', 'Mafia', 'Murder/Mystery', 'Horror']

alegría: ['Abstract Strategy', 'Adventure', 'Animals', 'Bluffing', 'Card Game', "Children's Game", 'City Building', 'Collectible Components', 'Economic', 'Educational', 'Environmental', 'Exploration', 'Fantasy', 'Farming', 'Humor', 'Memory', 'Miniatures', 'Party Game', 'Puzzle', 'Racing', 'Real-time', 'Science Fiction', 'Space Exploration', 'Spies/Secret Agents', 'Sports', 'Trivia', 'Video Game Theme', 'Zombies']

tristeza: ['Aviation / Flight', 'Age of Reason', 'Ancient', 'Arabian', 'Deduction', 'Dice', 'Electronic', 'Expansion for Base-game', 'Fighting', 'Game System', 'Math', 'Mature / Adult', 'Maze', 'Medical', 'Medieval', 'Mythology', 'Napoleonic', 'Nautical', 'Negotiation', 'Novel-based', 'Number', 'Pike and Shot', 'Pirates', 'Post-Napoleonic', 'Prehistoric', 'Print & Play', 'Transportation'

In [ ]:
# Funcion para determinar la emocion predominante entre las categorias de cada juego de mesa

def emocion_mayoritaria(row, emotions_dict):
    conteo_emociones = {'alegría': 0, 'enojo': 0, 'tristeza': 0}

    categorias = ast.literal_eval(row['categories'])

    for categoria in categorias:
        for emocion, lista_categorias in emotions_dict.items():
            if categoria in lista_categorias:
                conteo_emociones[emocion] += 1

    return max(conteo_emociones, key=conteo_emociones.get)

for index, row in bgg_df.iterrows():
    bgg_df.at[index, 'emocion'] = emocion_mayoritaria(row, emotions_dict)

In [ ]:
bgg_df['emocion'].value_counts()

,count
emocion,
alegría,800
tristeza,146
enojo,54


In [ ]:
# Se crea una columna para concatenar la informacion seleccionada para luego generar recomendaciones
bgg_df['categories'] = bgg_df['categories'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)
bgg_df['mechanics'] = bgg_df['mechanics'].apply(lambda x: ' '.join(x) if isinstance(x, list) else x)

# Columna Combined para hacer los embedding con toda la informacion
bgg_df['Combined'] = bgg_df['game_name'] + " " + bgg_df['categories'] + " " + bgg_df['description'] + " " + bgg_df['mechanics']

In [ ]:
# Se guarda el dataframe como CSV para no volver a ejecutar el pre procesamiento
bgg_df.to_csv('bgg_finish.csv', index=False)

In [ ]:
modelo_bgg = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_bgg = modelo_bgg.encode(bgg_df['Combined'].tolist(), convert_to_tensor=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Peliculas

In [ ]:
# Dataset de peliculas IMDB

imdb_df = pd.read_csv('/content/drive/MyDrive/NLP/IMDB-Movie-Data.csv')
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Rank                1000 non-null   int64  
 1   Title               1000 non-null   object 
 2   Genre               1000 non-null   object 
 3   Description         1000 non-null   object 
 4   Director            1000 non-null   object 
 5   Actors              1000 non-null   object 
 6   Year                1000 non-null   int64  
 7   Runtime (Minutes)   1000 non-null   int64  
 8   Rating              1000 non-null   float64
 9   Votes               1000 non-null   int64  
 10  Revenue (Millions)  1000 non-null   float64
 11  Metascore           1000 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB


In [ ]:
# Generos unicos
imdb_df['Genre'].str.split(',').explode().unique()

array(['Action', 'Adventure', 'Sci-Fi', 'Mystery', 'Horror', 'Thriller',
       'Animation', 'Comedy', 'Family', 'Fantasy', 'Drama', 'Music',
       'Biography', 'Romance', 'History', 'Crime', 'Western', 'War',
       'Musical', 'Sport'], dtype=object)

In [ ]:
emotions_dict_imdb = {
    'alegría': [
        'Action', 'Adventure', 'Animation', 'Comedy', 'Family',
        'Fantasy', 'Musical', 'Sport'
    ],
    'enojo': [
        'Horror', 'Thriller', 'Crime', 'War', 'Western'
    ],
    'tristeza': [
        'Drama', 'Biography', 'Romance', 'History', 'Music', 'Sci-Fi', 'Mystery'
    ]
}

In [ ]:
# Funcion para determinar la emocion predominante entre las categorias de cada juego de mesa

def emocion_mayoritaria(row, emotions_dict):
    conteo_emociones = {'alegría': 0, 'enojo': 0, 'tristeza': 0}

    categorias = row['Genre'].split(',')

    for categoria in categorias:
        for emocion, lista_categorias in emotions_dict.items():
            if categoria in lista_categorias:
                conteo_emociones[emocion] += 1

    return max(conteo_emociones, key=conteo_emociones.get)

for index, row in imdb_df.iterrows():
    imdb_df.at[index, 'emocion'] = emocion_mayoritaria(row, emotions_dict_imdb)

In [ ]:
imdb_df['emocion'].value_counts()

,count
emocion,
alegría,521
tristeza,320
enojo,159


In [ ]:
# Se crea una columna para concatenar la informacion seleccionada para luego generar recomendaciones

imdb_df['Combined'] = imdb_df['Title'] + " " + imdb_df['Genre'] + " " + imdb_df['Description'] + " " + imdb_df['Director']

In [ ]:
# Se guarda el dataframe como CSV para no volver a ejecutar el pre procesamiento
imdb_df.to_csv('imdb_finish.csv', index=False)

In [ ]:
modelo_imdb = SentenceTransformer('all-MiniLM-L6-v2')
embeddings_imdb = modelo_imdb.encode(imdb_df['Combined'].tolist(), convert_to_tensor=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


# Libros

In [ ]:
# Webscrapping para generar dataset de libros gutenberg
def get_book(id):
  URL = f"https://www.gutenberg.org/ebooks/{id}"
  page = requests.get(URL)

  soup = BeautifulSoup(page.content, "html.parser")
  if(soup.find('h1').text == 'No ebook by that number.'):
    return None

  author = soup.find('a', {
      'itemprop': 'creator',
      'rel': 'marcrel:aut',
      'typeof': 'pgterms:agent'
  })

  title = soup.find('td', {
      'itemprop': 'headline',
  }).text

  summary = ''
  subjects = []
  for tr in soup.find_all('tr'):
    th = tr.find('th')
    if th != None:
      th = th.text
      if th == "Summary":
        summary = tr.find('td').text.replace("(This is an automatically generated summary.)", '')
      if th == 'Subject':
        subject = tr.find('td').text
        subjects.append(subject)

  return {
      'author': author.text if author != None else 'anonymous',
      'summary': summary,
      'subject': ' '.join(subjects)
  }

In [ ]:
# Webscrapping para generar dataset de libros gutenberg

books = []
for id in range(1, 1001):
  book = get_book(id)
  if book is not None:
    books.append(book)

In [ ]:
book_df = pd.DataFrame(books)
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   982 non-null    object
 1   summary  982 non-null    object
 2   subject  982 non-null    object
dtypes: object(3)
memory usage: 23.1+ KB


In [ ]:
# Se guarda el dataframe como CSV
book_df.to_csv('books.csv', index=False)

In [ ]:
book_df = pd.read_csv('/content/drive/MyDrive/NLP/books.csv')
book_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 982 entries, 0 to 981
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   author   982 non-null    object
 1   summary  966 non-null    object
 2   subject  982 non-null    object
dtypes: object(3)
memory usage: 23.1+ KB


In [ ]:
book_df.head()

,author,summary,subject
0,"Jefferson, Thomas, 1743-1826","\n""The Declaration of Independence of the Unit...","\n\nUnited States -- History -- Revolution, 17..."
1,United States,"\n""The United States Bill of Rights"" by United...",\n\nCivil rights -- United States -- Sources\n...
2,"Kennedy, John F. (John Fitzgerald), 1917-1963","\n""John F. Kennedy's Inaugural Address"" by Joh...",\n\nUnited States -- Foreign relations -- 1961...
3,"Lincoln, Abraham, 1809-1865","\n""Lincoln's Gettysburg Address"" by Abraham Li...",\n\nConsecration of cemeteries -- Pennsylvania...
4,United States,"\n""The United States Constitution"" by United S...",\n\nUnited States -- Politics and government -...


In [ ]:
import re

# Función para extraer el título entre comillas de una cadena
def extraer_titulo(summary):
    if isinstance(summary, str):
        match = re.search(r'"(.*?)"', summary)
        return match.group(1) if match else None
    return None

# Aplicar la función a la columna 'summary' para crear la nueva columna 'titulo'
book_df['titulo'] = book_df['summary'].apply(extraer_titulo)


In [ ]:
# Creo categorias con palabras clave en ingles
categorias = {
    "historia": ["Revolution", "History", "War", "Independence", "Politics"],
    "romance": ["Love", "Romantic", "Affair", "Relationship", "Juvenile"],
    "terror": ["Horror", "Scary", "Ghost", "Fear", "Monster"],
    "ciencia ficción": ["Future", "Aliens", "Space", "Science", "Technology", "Fiction"],
    "biografía": ["Biography", "Life", "Memoir", "Autobiography"],
    "fantasía": ["Magic", "Fantasy", "Wizard", "Dragon", "Kingdom"],
    "filosofía": ["Philosophy", "Ethics", "Morality", "Existential", "Logic"],
    "psicología": ["Psychology", "Mind", "Behavior", "Mental", "Cognitive"],
    "arte": ["Art", "Painting", "Sculpture", "Museum", "Gallery"],
    "religión": ["Religion", "Spiritual", "Church", "God", "Faith"],
    "poesía": ["Poetry", "Poem", "Verse", "Sonnet"],
    "educación": ["Education", "Learning", "Teaching", "School", "Internet", "Terminology", "machines", "Pedagogy"]
}


# Función para clasificar en géneros, si no coincide nada ponemos "desconocido"
def clasificar_genero(text):
    for categoria, palabras in categorias.items():
        if any(palabra.lower() in text.lower() for palabra in palabras):
            return categoria
    return "Desconocido"

# Aplicar la función a cada fila
book_df['genero'] = book_df['subject'].apply(clasificar_genero)


In [ ]:
import spacy
import pandas as pd

nlp = spacy.load("en_core_web_sm")

In [ ]:
# Para definir los generos que no pudimos todavia vamos a utilizar ner y analizar las categorias

def extract_category(text):
    doc = nlp(text)
    categories = set(ent.label_ for ent in doc.ents)
    return categories

# Aplicar la función a cada fila del DataFrame
book_df['categories'] = book_df['subject'].apply(extract_category)



In [ ]:
# Cuantos desconocidos tenemos
desconocidos = book_df['genero'].value_counts().get("Desconocido", 0)

print(f"Número de libros clasificados como 'Desconocido': {desconocidos}")

Número de libros clasificados como 'Desconocido': 215


In [ ]:
# sistema para indcar segun que categoria nos devuelve ner un genero
for index, row in book_df.iterrows():
    if row['genero'] == "Desconocido":
      if "DATE" in row['categories']:
        book_df.at[index, 'genero'] = "historia"
      if "PERSON" in row['categories']:
        book_df.at[index, 'genero'] = "biografía"
      if "NORP" in row['categories']:
        book_df.at[index, 'genero'] = "educación"
      if "GPE" in row['categories']:
        book_df.at[index, 'genero'] = "desarrollo geográfico/político"
      if "ORG" in row['categories']:
        book_df.at[index, 'genero'] = "organizaciones"
      if "WORK_OF_ART" in row['categories']:
        book_df.at[index, 'genero'] = "arte"
      if "LOC" in row['categories']:
        book_df.at[index, 'genero'] = "desarrollo geográfico/político"
      if "CARDINAL" in row['categories']:
        book_df.at[index, 'genero'] = "economia"
      if "LANGUAGE" in row['categories']:
        book_df.at[index, 'genero'] = "diccionario"
      else:
        book_df.at[index, 'genero'] = "interes general"

In [ ]:
# cuanto queda de cada cosa
def contar_categorias(df):
    conteo = book_df['genero'].value_counts().to_dict()
    return conteo

conteo_genero = contar_categorias(book_df)
print(conteo_genero)

{'ciencia ficción': 299, 'interes general': 185, 'historia': 165, 'romance': 117, 'poesía': 60, 'biografía': 59, 'diccionario': 30, 'terror': 18, 'filosofía': 16, 'educación': 11, 'religión': 10, 'arte': 5, 'fantasía': 4, 'psicología': 3}


In [ ]:
book_df['subject'] = book_df['subject'].str.replace(r'\n', '', regex=True)
book_df['summary'] = book_df['summary'].str.replace(r'\n', '', regex=True)

In [ ]:
# Función para asignar emociones basadas en el género
def asignar_emocion(genero):
    if genero in ["terror", "historia", "ciencia ficción", "interés general"]:
        return "enojo"
    elif genero in ["biografía", "filosofía", "psicología", "religión"]:
        return "tristeza"
    else:
        return "alegría"

# Aplicar la función a cada fila para crear la nueva columna 'emocion'
book_df['emocion'] = book_df['genero'].apply(asignar_emocion)


In [ ]:
book_df['emocion'].value_counts()

,count
emocion,
enojo,482
alegría,412
tristeza,88


In [ ]:
# creamos el modelo en base a
book_df['Combined'] = book_df['author'] + " " + book_df['summary'] + " " + book_df['subject']

In [ ]:
# Se guarda el dataframe como CSV en su formato final, para evitar pre procesamiento
book_df.to_csv('books_finish.csv', index=False)

In [ ]:
modelo_libros = SentenceTransformer('all-MiniLM-L6-v2')
embedding_libros = modelo_libros.encode(book_df['Combined'].tolist(), convert_to_tensor=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
